# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/jovyan/work


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['193.18.0.2'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute(
    """
    CREATE KEYSPACE IF NOT EXISTS my_keyspace 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 3 }
    AND DURABLE_WRITES = true;
    """
    )
except Exception as e: 
    print('error: ',e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.execute("USE my_keyspace;")
except Exception as e: 
    print('error: ',e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
try:
    session.execute("DROP TABLE IF EXISTS song_by_sessionid_iteminsession;")
    session.execute(
    """
    CREATE TABLE song_by_sessionid_iteminsession(
    sessionId INT,
    itemInSession INT,
    artist VARCHAR,
    song VARCHAR,
    length DOUBLE,
    PRIMARY KEY (sessionId, itemInSession)
    );
    """
    )
except Exception as e: 
    print('error: ',e)

In [9]:
file = 'event_datafile_new.csv'

df = pd.read_csv(file)

for row in df.itertuples():
    query = """
    INSERT INTO song_by_sessionid_iteminsession(
    sessionId,
    itemInSession,
    artist,
    song,
    length
    )
    VALUES(%s,%s,%s,%s,%s)
    """
    session.execute(
        query, 
        (
            row.sessionId,
            row.itemInSession,
            row.artist,
            row.song,
            row.length
        ))

In [10]:
try:
    rows = session.execute(
    """
    select
        artist,
        song,
        length
    from song_by_sessionid_iteminsession
    where sessionId = 338
    AND itemInSession = 4
    """
    )
except Exception as e: 
    print('error: ',e)
rows_df = pd.DataFrame(rows)
rows_df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


This query retrieves the `artist`, `song` title, and `length` of a specific track 
from the table `song_by_sessionid_iteminsession`. The `WHERE` clause filters 
the data to return the song that was played during `sessionId` 338 and 
at the 4th item in that session. 

The query result shows that the song **"Music Matters (Mark Knight Dub)"** by **Faithless**, 
with a length of **495.3073 seconds**, was played in this session at the specified postion.


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
try:
    session.execute("DROP TABLE IF EXISTS song_by_userid_sessionid;")
    session.execute(
    """
    CREATE TABLE song_by_userid_sessionid(
    userId INT,
    sessionId INT,
    itemInSession INT,
    artist VARCHAR,
    song VARCHAR,
    firstName VARCHAR,
    lastName VARCHAR,
    PRIMARY KEY ((userId,sessionId), itemInSession)
    );
    """
    )
except Exception as e: 
    print('error: ',e)

In [12]:
file = 'event_datafile_new.csv'

df = pd.read_csv(file)

for row in df.itertuples():
    query = """
    INSERT INTO song_by_userid_sessionid(
    userId,
    sessionId,
    itemInSession,
    artist,
    song,
    firstName,
    lastName
    )
    VALUES(%s,%s,%s,%s,%s,%s,%s)
    """
    session.execute(
        query, 
        (
            row.userId,
            row.sessionId,
            row.itemInSession,
            row.artist,
            row.song,
            row.firstName,
            row.lastName
        ))           

In [13]:
try:
    rows = session.execute(
    """
    select
        artist,
        song,
        firstName,
        lastName
    from song_by_userid_sessionid
    where userId = 10 
    and sessionId = 182
    order by itemInSession
    """
    )
except Exception as e: 
    print('error: ',e)
rows_df = pd.DataFrame(rows)
rows_df

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


This query selects the `artist`, `song`, `firstName`, and `lastName` columns 
from the table `song_by_userid_sessionid`. The `WHERE` clause filters the data 
to retrieve all songs played by the user with `userId` 10 during `sessionId` 182. 
The results are ordered by `itemInSession`, indicating the sequence in which the songs were played during the session.

The query result shows that the user **Sylvie Cruz** listened to the following songs in session 182:
1. **"Keep On Keepin' On"** by **Down To The Bone**
2. **"Greece 2000"** by **Three Drives**
3. **"Kilometer"** by **Sebastien Tellier**
4. **"Catch You Baby (Steve Pitron & Max Sanna Radio Edit)"** by **LonnieGordon**


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
try:
    session.execute("DROP TABLE IF EXISTS user_by_song;")
    session.execute(
    """
    CREATE TABLE user_by_song(
    userId INT,
    song VARCHAR,
    firstName VARCHAR,
    lastName VARCHAR,
    PRIMARY KEY (song,userId)
    );
    """
    )
except Exception as e: 
    print('error: ',e)

In [15]:
file = 'event_datafile_new.csv'

df = pd.read_csv(file)

for row in df.itertuples():
    query = """
    INSERT INTO user_by_song(
    userId,
    song,
    firstName,
    lastName
    )
    VALUES(%s,%s,%s,%s)
    """
    session.execute(
        query, 
        (
            row.userId,
            row.song,
            row.firstName,
            row.lastName
        ))

In [16]:
try:
    rows = session.execute(
    """
    select
        firstName,
        lastName
    from user_by_song
    where song = 'All Hands Against His Own'
    """
    )
except Exception as e: 
    print('error: ',e)
rows_df = pd.DataFrame(rows)
rows_df

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


This query selects the `firstName` and `lastName` of users from the table `user_by_song` 
who listened to the song titled **"All Hands Against His Own"**. The `WHERE` clause filters 
the data to return only those users who have played this specific song.

The query result shows that the following users have listened to **"All Hands Against His Own"**:
1. **Jacqueline Lynch**
2. **Tegan Levine**
3. **Sara Jonson**


### Drop the tables before closing out the sessions

In [17]:
## TO-DO: Drop the keyspace before closing out the sessions
try:
    session.execute("DROP KEYSPACE my_keyspace;")
except Exception as e: 
    print('error: ',e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()